In [1]:
import pandas as pd
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

C:\Users\GLw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['HF_TOKEN'] = "aa993554a94a10b9022c4dc550df91e4470e66f2"

In [3]:
df = pd.read_csv("data/final_data.csv")

In [4]:
df.replace(to_replace=r'\n', value='', regex=True, inplace=True)
df.replace(to_replace=r'[\*\#]', value='', regex=True, inplace=True)
df.head()

,original_text,prompt,rewritten_text
0,"Hollis Price (born October 29, 1979) is an Ame...","Convert the text into a recipe, where each ins...",Hollis Price's Basketball DreamIngredients:- 1...
1,Grangewoood Independent School is an independe...,Transform the text into a cryptic crossword pu...,Across1. A school situated in a London borough...
2,Tap... tap... tap. I could hear him hiding som...,Translate the essence of this text into a horr...,"A tap, tap, tap echoed through the walls, send..."
3,A man smiled as he saw a package underneath hi...,Write like Oscar Wilde: Adopt Wilde's witty an...,"With a groan, he hoisted the box, an embodimen..."
4,After weeks of making phone calls we finally f...,Rewrite this text in the style of a time trave...,Weeks of frantic phone calls yielded a celesti...


In [5]:
df = df.sample(frac = 1)
df.head(5)

,original_text,prompt,rewritten_text
1980,Yep. I just signed up for nano ( this is my fi...,Convey the same message as this text but throu...,"You wanna get ahead of the curve, you gotta pl..."
2599,"Ok, still trying to get my head around Kaggle ...",Mold into a tagline for a movie.,It includes various changes made to the origin...
2689,we found many different ways to exploit the fr...,"Debate the key points of this text, presenting...",Arguments for: Listing frequency: The text pro...
3392,My daughter has been out over three hours past...,Roaring Twenties Flapper: Infuse the narrative...,My daughter has been out past her curfew for h...
3691,"Dear God, I thought as the removed submission ...",Street-Smart Urbanite: Use the savvy and sharp...,The removed submission notification flashed ac...


In [6]:
def add_special_tokens(df):
    # Concatenate the prompts and texts with a special token in between
    df['combined_text'] = "Original text: "+ df['original_text'] + " <|endoftext|> Rewritten text:" + df['rewritten_text'] + "<|endoftext|> The prompt used to generate rewritten text given original text is: " + df['prompt']
    df['combined_text_inf'] = "Original text: " + df['original_text'] + " <|endoftext|> Rewritten text: " + df['rewritten_text'] + "<|endoftext|> The prompt used to generate rewritten text given original text is: "
    return df

df = add_special_tokens(df)

In [7]:
data_file = 'fine_tune_data.txt'
df['combined_text'][:1000].to_csv(data_file, index=False, header=False)


In [8]:
data_file = 'fine_tune_data.txt'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('gpt2-large')

# Ensure the tokenizer will not split <|endoftext|> into multiple tokens
tokenizer.add_special_tokens({'additional_special_tokens': ['<|endoftext|>']})
model.resize_token_embeddings(len(tokenizer))

# 3. Create the dataset and data collator
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=data_file,
    block_size=128)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False)

C:\Users\GLw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [9]:

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    overwrite_output_dir=True,       # overwrite the content of the output directory
    num_train_epochs=10,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    save_steps=10_400,               # after # steps model is saved
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# 5. Start the fine-tuning
trainer.train()

C:\Users\GLw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lgan (cse151b_llm). Use `wandb login --relogin` to force relogin


 14%|█▎        | 501/3710 [02:01<12:52,  4.15it/s]

{'loss': 2.786, 'grad_norm': 2.198972225189209, 'learning_rate': 4.326145552560647e-05, 'epoch': 1.35}


 27%|██▋       | 1001/3710 [04:04<11:32,  3.91it/s]

{'loss': 1.836, 'grad_norm': 2.6028943061828613, 'learning_rate': 3.652291105121294e-05, 'epoch': 2.7}


...

 94%|█████████▍| 3501/3710 [14:16<00:50,  4.14it/s]

{'loss': 0.2059, 'grad_norm': 1.1882586479187012, 'learning_rate': 2.830188679245283e-06, 'epoch': 9.43}


100%|██████████| 3710/3710 [15:07<00:00,  4.09it/s]

{'train_runtime': 909.6077, 'train_samples_per_second': 32.607, 'train_steps_per_second': 4.079, 'train_loss': 0.9940205463501963, 'epoch': 10.0}


TrainOutput(global_step=3710, training_loss=0.9940205463501963, metrics={'train_runtime': 909.6077, 'train_samples_per_second': 32.607, 'train_steps_per_second': 4.079, 'train_loss': 0.9940205463501963, 'epoch': 10.0})

In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('model/gpt2/')
tokenizer = GPT2Tokenizer.from_pretrained('model/gpt2/')

In [11]:
model.save_pretrained('model/gpt2/')
tokenizer.save_pretrained('model/gpt2/')

('model/gpt2/tokenizer_config.json',
 'model/gpt2/special_tokens_map.json',
 'model/gpt2/vocab.json',
 'model/gpt2/merges.txt',
 'model/gpt2/added_tokens.json')

In [12]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
model= model.to(device)

In [14]:
vdf = df.iloc[:4000]

In [15]:
row_index = 40
original = vdf.iloc[row_index, 1]
rewritten = vdf.iloc[row_index, 3]
prompt = vdf.iloc[row_index, 2]

In [16]:
print("original: ",original)
print("prompt: ",prompt)
print("rewritten: ",rewritten)

original:  Convert it into a narrative from a sentient AI's perspective.
prompt:  The echoes of Casey's cleats carving through the earth and the roar of the crowd reverberate through the depths of my consciousness. A formidable runner with a heart of gold, he commanded the Winnipeg Blue
rewritten:  Original text: Thomas Ray Casey, better known as Tom "Citation" Casey (July 30, 1924 – October 10, 2002) played for the Winnipeg Blue Bombers of the Western Interprovincial Football Union from 1950 to 1956, during which time he led the league in rushing yards and was named a divisional all-star each year.  He was elected into the Canadian Football Hall of Fame in 1964, the first African American to be inducted.  He also played one year for the Hamilton Wildcats in 1949.  He was named to the All-Time Blue Bomber Greats 75th Anniversary team. <|endoftext|> Rewritten text:The echoes of Casey's cleats carving through the earth and the roar of the crowd reverberate through the depths of my consci

In [17]:
target = []
predict = []
num = 40
for i in range(1):
    original = vdf.iloc[num, 1]
    rewritten = vdf.iloc[num, 3]
    prompt = vdf.iloc[num, 2]


    input_text = "Original text: "+ original + "<|endoftext|> " + "Rewritten text: " + rewritten + "<|endoftext|> What was the prompt used to rewrite original text into rewritten text? <|prompt|> "  # This is your input text
    #input_text = "What is one plus one"
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    # Generate text using the model
    # Adjust the max_length parameter as needed
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=len(input_text) + 20,  # Adjust the max length of the output text if needed
        temperature=1.0,  # Temperature controls the randomness of the output
        top_k=50,  # Top-k sampling
        top_p=0.95,  # Nucleus sampling
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1,  # Number of output sequences to generate
    )

    # Decode the output
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=False)

    



    #print("Generated text:", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


RuntimeError: The size of tensor a (1024) must match the size of tensor b (1025) at non-singleton dimension 3

In [ ]:
input_text

'Original text: I love R but using Python thus far for this competition, this is a great R kernel, thanks.<|endoftext|> Rewritten text: "I love R, but I\'ve been using Python thus far for this competition. This is a great R kernel, thanks. It\'s like a breath of fresh air in the midst of a data wrangling battle."<|endoftext|> What was the prompt used to rewrite original text into rewritten text? <|prompt|> '

In [ ]:
prompt

'Recast this as a gripping saga.'

In [ ]:
generated_text

'Original text: I love R but using Python thus far for this competition, this is a great R kernel, thanks.<|endoftext|> Rewritten text: "I love R, but I\'ve been using Python thus far for this competition. This is a great R kernel, thanks. It\'s like a breath of fresh air in the midst of a data wrangling battle."<|endoftext|> What was the prompt used to rewrite original text into rewritten text?  <|prompt|>   `` Its so nice,\'\' my dear friend told me as he inserted his crayons directly above our class photo on the wall while we talked during lunch break!\'\' He paused and looked at us blankly before glancing over with an enormous grin upon his face - which really did not look fair compared to what everyone else was looking forward- when all that information finally came back and they were approved by the DMAs they could see clearly where it went! Everyone loved what they saw. We now knew everything that was real; information that whoever had worked within AI program proved very clearl

In [ ]:
generated_text

a = generated_text.split("<|endoftext|>")[3]
a


'Expand on the mystical elements present in the rewritten text.'

In [ ]:
from sentence_transformers import SentenceTransformer

scs_model = SentenceTransformer("sentence-t5-base")

In [ ]:
def  batch_cosine_similarity(x1, x2):

    # Normalize the embeddings to unit vectors
    x1_norm = torch.nn.functional.normalize(x1, p=2, dim=-1)
    x2_norm = torch.nn.functional.normalize(x2, p=2, dim=-1)
    
    # Compute the cosine similarity
    cos_sim = torch.mm(x1_norm, x2_norm.transpose(0, 1))
    
    return cos_sim

def sharpened_cosine_similarity_batch(scs_model, output_texts, target_texts, sharpen_factor=3):
    # Assuming scs_model.encode() returns PyTorch tensors
    target_embeddings = scs_model.encode(target_texts, convert_to_tensor=True)
    output_embeddings = scs_model.encode(output_texts, convert_to_tensor=True)
    
    # Calculate batch cosine similarities using the previously defined batch_cosine_similarity function
    cos_sims = batch_cosine_similarity(target_embeddings, output_embeddings)
    
    # Apply the sharpening factor to each similarity score and create a list of tensors
    sharpened_scores = [cos_sims[i][i].unsqueeze(0) ** sharpen_factor for i in range(cos_sims.size(0))]
    
    return sharpened_scores


In [ ]:
a=[" [_\'Sorrowful thoughts,\' written with an editor\'s care ] ( /r/WritingPrompts )"]
b=[prompt]

In [ ]:
prompt

'Reshape this as an absorbing narrative.'

In [ ]:
sharpened_cosine_similarity_batch(scs_model, a, b, sharpen_factor=3)


[tensor([0.4557], device='cuda:0')]